In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pymongo
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [42]:
nasa_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
mw_url = 'https://twitter.com/marswxreport?lang=en'
mf_url = 'https://space-facts.com/mars/'
mh_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [43]:
nasa_resp = requests.get(nasa_url)
jpl_resp = requests.get(jpl_url)
mw_resp = requests.get(mw_url)
mf_resp = requests.get(mf_url)
mh_resp = requests.get(mh_url)

# NASA Mars News

In [59]:
##NASA MARS NEWS
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
#Assign the text to variables that you can reference later.

n_soup = bs(nasa_resp.text,'html.parser')



title= n_soup.find_all('div',class_='content_title')
news_title = title[0].text
#print("TITLE", news_title)

para = n_soup.find_all('div',class_='rollover_description_inner')   
news_para = para[0].text
#print("PARAGRAPH",newsp)    

news = {'paragraph':news_para,'title':news_title}

latest = []

latest.append(news)

latest


[{'paragraph': "\nIt's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home. \n",
  'title': '\n\nOpportunity Hunkers Down During Dust Storm\n\n'}]

# JPL Mars Space Images - Featured Image

In [7]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image and 
#assign the url string to a variable called featured_image_url.
#Make sure to find the image url to the full size .jpg image.
#Make sure to save a complete url string for this image.
browser.visit(jpl_url)
jpl_html = browser.html



j_soup = bs(jpl_html,'lxml')
#print(j_soup)

#link_img = j_soup.find('a',class_='button fancybox')

#link_img = j_soup.find('footer')

ft_img = j_soup.find('article', class_='carousel_item')['style'][23:75]

ft_img_url = jpl_url+ft_img
ft_img_url 
#link_img
#print(j_soup)

#link = link_img.find('a',class_='button fancybox',)
#print(link.href)

'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/wallpaper/PIA16105-1920x1200.jpg'

# Mars Weather Twitter

In [22]:
#Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. 
#Save the tweet text for the weather report as a variable called mars_weather.
mw_soup= bs(mw_resp.text,'lxml')
#print(mw_soup)

tweet = mw_soup.find_all('div',class_="js-tweet-text-container")

mars_weather = tweet[0].text
mars_weather

'\nSol 2147 (2018-08-21), high -15C/5F, low -68C/-90F, pressure at 8.70 hPa, daylight 05:30-17:44\n'

# Mars Facts

In [9]:
#Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including
#Diameter, Mass, etc.
#Use Pandas to convert the data to a HTML table string.

fact_tbl = pd.read_html(mf_url)
fact_tbl

df_tbl = fact_tbl[0]
df_tbl

#type(fact_tbl)

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [10]:
df_tbl.columns = ["Parameters", "Measurements"]
df_tbl

,Parameters,Measurements
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


# Mars Hemispheres

In [11]:
# Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing
# the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
# Append the dictionary with the image url string and the hemisphere title to a list.
# This list will contain one dictionary for each hemisphere.

In [21]:
mh_soup = bs(mh_resp.text,'lxml')
    
hem_dic = {}
names = []

myList = []

baseurl = 'https://astrogeology.usgs.gov'
    
items = mh_soup.find_all('div', class_='item')

for i in items:
    titles = i.find('img')['alt']
    
    hem_dic={'title':titles}
    
    myList.append(hem_dic)
    
    links = i.find('a', href=True)
    
    found = links['href']
    
    url = baseurl + found
    
    browser.visit(url)
        
    dwn_soup = bs(browser.html,'lxml')
        
    dwn_section = dwn_soup.find('div', class_='downloads')
        
    img_links = dwn_section.find('a', href=True)
        
    img_url = img_links['href']
    
    hem_dic = {'img_url' : img_url}
    
    myList.append(hem_dic)
    
myList


[{'title': 'Cerberus Hemisphere Enhanced thumbnail'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced thumbnail'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced thumbnail'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced thumbnail'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]